In [5]:
import pandas as pd

df_Abstract = pd.read_csv('final-dataset(A1-31306samples)-train-topicmodel.csv')
df_Abstract.head()

,publication_number,country_code,kind_code,title,abstract,claims,publication_date,ipc_code,cpc_code,first_claim,claim_lengths,sub_classes,sub_class,abstract_lengths
0,US2020097067A1,US,A1,Artificial Intelligence System and Interactive...,"A reality interactive responding system, compr...","1 . A reality interactive responding system, c...",20200326,G06F9/448,G16H40/67,"1 . A reality interactive responding system, c...",200,"['G06F9/448', 'G16H40/67']","['G06F', 'G16H']",123
1,US2020098473A1,US,A1,Data Storage and Retrieval System for Non-Cont...,A web-based interface enables medical personne...,What is claimed is: \n \n 1 . A da...,20200326,G16H40/67,H04L67/1097,What is claimed is: 1 . A data storage and ret...,200,"['G16H40/67', 'H04L67/1097']","['G16H', 'H04L']",142
2,US2020098451A1,US,A1,Hybrid analysis framework for prediction of ou...,A facility for predicting patient outcomes on ...,"1 . A method in a computing system, comprising...",20200326,G16H10/20,G16H10/20,"1 . A method in a computing system, comprising...",200,"['G16H10/20', 'G16H10/20']","['G16H', 'G16H']",81
3,US2020098458A1,US,A1,Medical cannabis platform with physician and p...,"Through a physician&#39;s portal, a platform c...",What is claimed is: \n \n 1 . A me...,20200326,G16H80/00,A61K36/185,What is claimed is: 1 . A method for providing...,200,"['G16H80/00', 'A61K36/185']","['G16H', 'A61K']",148
4,US2020093988A1,US,A1,Patient day planning systems and methods,"Infusion systems, infusion devices, and relate...",What is claimed is: \n \n 1 . A me...,20200326,G16H20/17,A61M2230/201,What is claimed is: 1 . A method of monitoring...,200,"['G16H20/17', 'A61M2230/201']","['G16H', 'A61M']",110


In [6]:
len(df_Abstract)

31306

In [7]:
df_Abstract.head()

,publication_number,country_code,kind_code,title,abstract,claims,publication_date,ipc_code,cpc_code,first_claim,claim_lengths,sub_classes,sub_class,abstract_lengths
0,US2020097067A1,US,A1,Artificial Intelligence System and Interactive...,"A reality interactive responding system, compr...","1 . A reality interactive responding system, c...",20200326,G06F9/448,G16H40/67,"1 . A reality interactive responding system, c...",200,"['G06F9/448', 'G16H40/67']","['G06F', 'G16H']",123
1,US2020098473A1,US,A1,Data Storage and Retrieval System for Non-Cont...,A web-based interface enables medical personne...,What is claimed is: \n \n 1 . A da...,20200326,G16H40/67,H04L67/1097,What is claimed is: 1 . A data storage and ret...,200,"['G16H40/67', 'H04L67/1097']","['G16H', 'H04L']",142
2,US2020098451A1,US,A1,Hybrid analysis framework for prediction of ou...,A facility for predicting patient outcomes on ...,"1 . A method in a computing system, comprising...",20200326,G16H10/20,G16H10/20,"1 . A method in a computing system, comprising...",200,"['G16H10/20', 'G16H10/20']","['G16H', 'G16H']",81
3,US2020098458A1,US,A1,Medical cannabis platform with physician and p...,"Through a physician&#39;s portal, a platform c...",What is claimed is: \n \n 1 . A me...,20200326,G16H80/00,A61K36/185,What is claimed is: 1 . A method for providing...,200,"['G16H80/00', 'A61K36/185']","['G16H', 'A61K']",148
4,US2020093988A1,US,A1,Patient day planning systems and methods,"Infusion systems, infusion devices, and relate...",What is claimed is: \n \n 1 . A me...,20200326,G16H20/17,A61M2230/201,What is claimed is: 1 . A method of monitoring...,200,"['G16H20/17', 'A61M2230/201']","['G16H', 'A61M']",110


In [8]:
first_record = df_Abstract['abstract'][1]
first_record

'A web-based interface enables medical personnel to remotely monitor medical devices. A monitoring system records operational data and alarms from the medical devices in a file. However, since network connections between the medical devices and the monitoring system are intermittent, the file does not contain a contiguous stream of data for each medical device. The file pauses recording during gaps in network connectivity. The system displays current data, as well as a list of alarms. If medical personnel wish to view more detail about an earlier time or one of the alarms, the system calculates where in the file the medical device data was recorded. This calculation accounts for the discontiguous nature of the data. The system uses times the network connection is made and broken to calculate an index into the file that corresponds to the time of the user-selected alarm.'

In [9]:
import torch
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the Sentence Transformers model
sentence_model = SentenceTransformer('AI-Growth-Lab/PatentSBERTa')
#sentence_model = SentenceTransformer("all-MiniLM-L6-v2", device=device)

# Adjust UMAP hyperparameters
umap_model = UMAP(n_neighbors=3, 
                  n_components=3, 
                  min_dist=0.05, 
                  metric='cosine', 
                  random_state=100,
                  n_jobs=-1)  # Utilize all available CPU cores for UMAP preprocessing

# Adjust HDBSCAN hyperparameters
hdbscan_model = HDBSCAN( min_cluster_size=80,
                        min_samples=40, 
                        metric='euclidean', 
                        cluster_selection_method='eom', 
                        prediction_data=True)

# Adjust CountVectorizer hyperparameters
vectorizer_model = CountVectorizer(ngram_range=(1, 3), min_df=10, max_df=0.5)


# c-TF-IDF
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

# Run Model
topic_model = BERTopic(umap_model=umap_model,
                       embedding_model=sentence_model, 
                       hdbscan_model=hdbscan_model, 
                       vectorizer_model=vectorizer_model,
                       ctfidf_model=ctfidf_model,
                       verbose=True)

# Replace 'df['first_claim']' with your data source for the first_claim column
topics, probabilities = topic_model.fit_transform(df_Abstract['abstract'])


Batches:   0%|          | 0/979 [00:00<?, ?it/s]

2023-09-29 10:47:32,037 - BERTopic - Transformed documents to Embeddings
2023-09-29 10:48:22,850 - BERTopic - Reduced dimensionality
2023-09-29 10:48:25,050 - BERTopic - Clustered reduced embeddings


In [10]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

documents = pd.DataFrame({"Document": df_Abstract['abstract'],
                          "ID": range(len(df_Abstract['abstract'])),
                          "Topic": topics})

documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] for topic in range(len(set(topics))-1)]


# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_npmi')
coherence_c_nmpi = coherence_model.get_coherence()
print("c_npmi is: ",coherence_c_nmpi)

c_npmi is:  -0.1650835367995478


In [11]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

documents = pd.DataFrame({"Document":df_Abstract['abstract'],
                          "ID": range(len(df_Abstract['abstract'])),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence_CV= coherence_model.get_coherence()
print("C_V is: ", coherence_CV)

C_V is:  0.35482890908014697


In [12]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

documents = pd.DataFrame({"Document":df_Abstract['abstract'],
                          "ID": range(len(df_Abstract['abstract'])),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='u_mass')
coherence_u_mass= coherence_model.get_coherence()
print("u_mass is: ", coherence_u_mass)

u_mass is:  -0.5687047678531202


In [13]:
 topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,16163,-1_sleep_bed_claim_emr,"[sleep, bed, claim, emr, claims, eye, transact...",[An image-guided tool and method for ophthalmi...
1,0,2160,0_the medical image_medical images_image proce...,"[the medical image, medical images, image proc...",[Methods and systems for automatically analyzi...
2,1,1675,1_disclosure relates_disclosed are_disclosure ...,"[disclosure relates, disclosed are, disclosure...",[Disclosed are methods comprising: receiving d...
3,2,1156,2_the invention relates_present invention is_r...,"[the invention relates, present invention is, ...",[The present invention discloses methods and a...
4,3,1128,3_referral_consumer_and health care_the consumer,"[referral, consumer, and health care, the cons...",[A computer-implemented method includes receiv...
5,4,1053,4_pharmacy_dispensing_the prescription_the med...,"[pharmacy, dispensing, the prescription, the m...",[Systems and methods are provided for facilita...
6,5,497,5_invention provide_present invention provide_...,"[invention provide, present invention provide,...",[Certain embodiments of the present invention ...
7,6,419,6_glucose_insulin_blood glucose_bolus,"[glucose, insulin, blood glucose, bolus, diabe...",[A controller for an insulin infusion device i...
8,7,383,7_ecg_heart rate_electrocardiogram_lead,"[ecg, heart rate, electrocardiogram, lead, ven...",[A monitoring device includes a housing config...
9,8,353,8_food_nutritional_diet_of food,"[food, nutritional, diet, of food, meal, nutri...",[This invention is a device and system for mon...


In [14]:
#adding topics and probs for eachdoc in dataset
df_Abstract['topics'] = topics
df_Abstract['prob'] = probabilities
df_Abstract_topic=df_Abstract[["publication_number","title","abstract","sub_classes","sub_class","topics","prob"]]
df_Abstract_topic

,publication_number,title,abstract,sub_classes,sub_class,topics,prob
0,US2020097067A1,Artificial Intelligence System and Interactive...,"A reality interactive responding system, compr...","['G06F9/448', 'G16H40/67']","['G06F', 'G16H']",3,1.000000
1,US2020098473A1,Data Storage and Retrieval System for Non-Cont...,A web-based interface enables medical personne...,"['G16H40/67', 'H04L67/1097']","['G16H', 'H04L']",10,0.788830
2,US2020098451A1,Hybrid analysis framework for prediction of ou...,A facility for predicting patient outcomes on ...,"['G16H10/20', 'G16H10/20']","['G16H', 'G16H']",-1,0.000000
3,US2020098458A1,Medical cannabis platform with physician and p...,"Through a physician&#39;s portal, a platform c...","['G16H80/00', 'A61K36/185']","['G16H', 'A61K']",3,1.000000
4,US2020093988A1,Patient day planning systems and methods,"Infusion systems, infusion devices, and relate...","['G16H20/17', 'A61M2230/201']","['G16H', 'A61M']",-1,0.000000
...,...,...,...,...,...,...,...
31301,US2016253489A1,User authentication system,A user authentication system performs user rec...,"['G16H10/60', 'G06F21/32']","['G16H', 'G06F']",-1,0.000000
31302,US2016253467A1,"Diagnosis support apparatus and method, and no...",A diagnosis support apparatus for diagnosis of...,"['G16H10/60', 'A61B5/743']","['G16H', 'A61B']",0,0.882754
31303,US2016253462A1,Novel open-access scheduling system that optim...,A patient appointment schedule is generated fo...,"['G16H40/20', 'G06F19/327']","['G16H', 'G06F']",-1,0.000000
31304,US2016249985A1,Interrelated point acquisition for navigated s...,The present invention relates to a method for ...,"['G16H20/40', 'G06F19/324']","['G16H', 'G06F']",-1,0.000000


# prediction

In [15]:
import pandas as pd

df_Abstract_test = pd.read_csv('test-queries-USPTO(A1)-2023-G16H.csv')
df_Abstract_test.head()

,publication_numbers,abstract,first_claim,class_codes
0,US20230238130A1,A physiological sensor has light emitting sour...,1. A physiological monitoring device comprisin...,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/..."
1,US20230270344A1,A wearable monitoring device includes a band c...,"1. A monitoring device, comprising:\na band co...","A61B5/02405,A61B5/01,A61B5/16,A61B5/02055,A61B..."
2,US20230200909A1,A number of improvements are provided relating...,1-25. (canceled) 26. A method for guiding a fr...,"A61B17/17,A61B2090/061,A61B90/06,A61B2090/365,..."
3,US20230218347A1,Embodiments include a system for determining c...,1-184. (canceled) 185. A computer-implemented ...,"G06V10/46,G06V20/698,G06T2207/20112,G06T7/13,A..."
4,US20230063013A1,A community based response system for providin...,1. (canceled) 2. A community based response sy...,"H04M1/72418,G08B,G08,G08B25/016,G,H04W4/023,G1..."


# test1

In [16]:
df_Abstract_test.iloc[0]


publication_numbers                                      US20230238130A1
abstract               A physiological sensor has light emitting sour...
first_claim            1. A physiological monitoring device comprisin...
class_codes            A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...
Name: 0, dtype: object

In [17]:
test1 = df_Abstract_test.loc[0, 'abstract']
test1

'A physiological sensor has light emitting sources, each activated by addressing at least one row and at least one column of an electrical grid. The light emitting sources are capable of transmitting light of multiple wavelengths and a detector is responsive to the transmitted light after attenuation by body tissue.'

In [18]:
import numpy as np 

# Find topics
num_of_topics = 5
similar_topics, similarity = topic_model.find_topics(test1, top_n=num_of_topics); 

# Print results
print(f'The top {num_of_topics} similar topics are {similar_topics}, and the similarities are {np.round(similarity,2)}')

for idx, topic_idx in enumerate(similar_topics):
    topic = topic_model.get_topic(topic_idx)
    keywords = ' '.join(str(keyword) for keyword in topic[0])
    print(f"Topic {idx+1}: {keywords}")

The top 5 similar topics are [40, 13, 10, 46, 7], and the similarities are [0.67 0.67 0.65 0.64 0.64]
Topic 1: biological information 0.6552945015816999
Topic 2: vital sign 0.3686265162369979
Topic 3: alarm 0.5276353125994339
Topic 4: wearable device 0.5503394171785392
Topic 5: ecg 0.6001228641181893


In [19]:
filter_topics_filter = df_Abstract_topic[df_Abstract_topic['topics'] == 43]
filter_topics_filter = filter_topics_filter.sort_values('prob', ascending=False)
filter_topics_filter

,publication_number,title,abstract,sub_classes,sub_class,topics,prob
19,US2020093457A1,Radiation system,The radiation system includes a radiation devi...,"['G16H30/20', 'G16H30/20']","['G16H', 'G16H']",43,1.000000
23486,US2010283583A1,"Radiation image capturing method, radiation im...","A radiation image capturing method, a radiatio...","['H04Q5/22', 'G16H40/63']","['H04Q', 'G16H']",43,1.000000
22952,US2010137148A1,System and Method for Radioisotope Identification,A system and method are providing for analyzin...,"['G16H50/30', 'G01V5/0008']","['G16H', 'G01V']",43,1.000000
22879,US2010121184A1,"Systems, methods and apparatus for preparation...","In one aspect, systems, methods and apparatus ...","['G16H20/13', 'G16H20/13']","['G16H', 'G16H']",43,1.000000
21451,US2014118618A1,"Transmission apparatus, transmission method, r...",There is provided a transmission apparatus inc...,"['G16H40/67', 'H04N7/18']","['G16H', 'H04N']",43,1.000000
...,...,...,...,...,...,...,...
10113,US2017245828A1,"Control apparatus, radiation imaging apparatus...",A control apparatus which receives a radiation...,"['G16H30/20', 'A61B6/563']","['G16H', 'A61B']",43,1.000000
10213,US2017273652A1,"Image processing device, radiographic imaging ...",The present disclosure provides an image proce...,"['A61B6/00', 'G16H50/30']","['A61B', 'G16H']",43,0.993916
10421,US2017316562A1,Determining at least one protocol parameter fo...,A method for determining at least one protocol...,"['A61B6/00', 'G16H50/30']","['A61B', 'G16H']",43,0.988982
19465,US2018325476A1,"Radiation imaging system, information processi...",A radiation imaging system includes a detector...,"['A61B6/00', 'G16H50/30']","['A61B', 'G16H']",43,0.965761


In [20]:
# Define the range of topic IDs you want to retrieve documents for
selected_topic_range =[43, 5, 49, 42, 16]  # Replace with your desired range of topic IDs

# Create an empty DataFrame to store the top 10 documents
top_10_documents_test1 = pd.DataFrame()

# Iterate through the selected topic IDs
for selected_topic_id in selected_topic_range:
    # Filter the dataset based on the current topic ID
    filtered_df = df_Abstract_topic[df_Abstract_topic['topics'] == selected_topic_id]
    
    # Sort the filtered dataset by similarity scores in descending order
    sorted_df = filtered_df.sort_values(by='prob', ascending=False)
    
    # Retrieve the top 10 documents for the current topic
    top_10_for_topic = sorted_df.head(10)
    
    # Append the top 10 documents for the current topic to the result DataFrame
    top_10_documents_test1 = top_10_documents_test1.append(top_10_for_topic)

# Display the top 10 documents for all selected topics
top_10_documents_test1

,publication_number,title,abstract,sub_classes,sub_class,topics,prob
19,US2020093457A1,Radiation system,The radiation system includes a radiation devi...,"['G16H30/20', 'G16H30/20']","['G16H', 'G16H']",43,1.0
23486,US2010283583A1,"Radiation image capturing method, radiation im...","A radiation image capturing method, a radiatio...","['H04Q5/22', 'G16H40/63']","['H04Q', 'G16H']",43,1.0
22952,US2010137148A1,System and Method for Radioisotope Identification,A system and method are providing for analyzin...,"['G16H50/30', 'G01V5/0008']","['G16H', 'G01V']",43,1.0
22879,US2010121184A1,"Systems, methods and apparatus for preparation...","In one aspect, systems, methods and apparatus ...","['G16H20/13', 'G16H20/13']","['G16H', 'G16H']",43,1.0
21451,US2014118618A1,"Transmission apparatus, transmission method, r...",There is provided a transmission apparatus inc...,"['G16H40/67', 'H04N7/18']","['G16H', 'H04N']",43,1.0
21124,US2014045656A1,Interactive Exercise Equipment with Exercise M...,An exercise equipment includes an equipment bo...,"['A63B24/00', 'G16H20/30']","['A63B', 'G16H']",43,1.0
21071,US2014029833A1,"X-ray imaging apparatus, method of controlling...",An X-ray imaging apparatus comprises an examin...,"['A61B5/00', 'G16H40/20']","['A61B', 'G16H']",43,1.0
20913,US2013336458A1,"Radiation imaging control apparatus, radiation...",A radiation imaging control apparatus includes...,"['G16H30/40', 'H05G1/30']","['G16H', 'H05G']",43,1.0
20623,US2019175270A1,Reusable transmission network for dividing ene...,Microwave ablation systems and methods use a c...,"['G16H40/63', 'H05B6/68']","['G16H', 'H05B']",43,1.0
19950,US2019046135A1,Image processing device and method for operati...,A detection unit of a CPU of a console detects...,"['G06K9/32', 'G16H50/30']","['G06K', 'G16H']",43,1.0


In [21]:
num_records = top_10_documents_test1.shape[0]
num_records

40

In [22]:
# # Define a custom function to format the text
# def format_text(text):
#     # Remove brackets and single quotes, split by comma and strip whitespace
#     cleaned_text = ''.join(text).replace('[', '').replace(']', '').replace("'", '').split(',')
#     # Filter out empty strings and strip whitespace
#     cleaned_text = [item.strip() for item in cleaned_text if item.strip()]
#     # Join the cleaned text with commas and spaces
#     formatted_text = ', '.join(cleaned_text)
#     return formatted_text

# # Apply the custom function to the 'sub_classes' and 'sub_class' columns
# top_10_documents_test1['combined_class'] = top_10_documents_test1.apply(lambda x: format_text(x['sub_classes']) + ', ' + format_text(x['sub_class']), axis=1)

# # Display the resulting DataFrame with the 'combined' column
# print(top_10_documents_test1[['sub_classes', 'sub_class', 'combined_class']])


In [23]:
# # Get the class_codes from the first record of df_Abstract_test and convert it to a string, then split it by comma
# class_codes_value = df_Abstract_test['class_codes'].iloc[0]
# if pd.notna(class_codes_value):
#     target_sub_classes = str(class_codes_value).split(',')
# else:
#     target_sub_classes = []

# # Define a function to check for matching classes
# def has_matching_class(row):
#     # Extract the first 4 letters from each code in class_codes and combined_class
#     target_codes = [code[:4] for code in target_sub_classes]
#     combined_codes = [code[:4] for code in str(row['combined_class']).split(',')]

#     # Check if there are any common codes
#     return any(code in combined_codes for code in target_codes)

# # Apply the function to create a boolean mask
# top_10_documents_test1['has_matching_class'] = top_10_documents_test1.apply(has_matching_class, axis=1)

# top_10_documents_test1

In [24]:
# top_10_documents_test1.shape[0]

# list of 100 queries

In [25]:
# Assuming you want to predict topics for the first 100 samples in 'df_Abstract_test'
num_samples_to_predict = 100

num_of_topics = 5
results = []

# Assuming you have a list of 100 queries in test
for query in df_Abstract_test['abstract'][:num_samples_to_predict]:
    similar_topics, similarity = topic_model.find_topics(query, top_n=num_of_topics)
    results.append((similar_topics, similarity))

# Now, the 'results' list contains the similar topics and similarities for each query
# You can access the results for a specific query like this:
for i, (similar_topics, similarity) in enumerate(results):
    print(f"Query {i + 1}: Similar Topics {similar_topics}, Similarity {similarity}")


Query 1: Similar Topics [40, 13, 10, 46, 7], Similarity [0.67092514, 0.66505456, 0.64937407, 0.6383254, 0.6357091]
Query 2: Similar Topics [13, 46, 39, 7, 10], Similarity [0.6222541, 0.6102503, 0.5832952, 0.57399035, 0.5695604]
Query 3: Similar Topics [39, 20, 3, -1, 0], Similarity [0.627025, 0.61962545, 0.6120764, 0.6087836, 0.60691154]
Query 4: Similar Topics [36, 5, 7, 41, 34], Similarity [0.8323266, 0.7833832, 0.7461008, 0.7445706, 0.736279]
Query 5: Similar Topics [13, 7, 10, 25, 26], Similarity [0.6255849, 0.6190642, 0.6053225, 0.6048085, 0.60412645]
Query 6: Similar Topics [13, 40, 10, 6, -1], Similarity [0.7210678, 0.7097218, 0.6871717, 0.68660045, 0.6714189]
Query 7: Similar Topics [11, 46, 20, -1, 31], Similarity [0.759723, 0.7257217, 0.67620224, 0.66556543, 0.66291046]
Query 8: Similar Topics [11, 46, 20, 13, 7], Similarity [0.7967641, 0.71084565, 0.70571953, 0.6802571, 0.67850447]
Query 9: Similar Topics [4, 30, -1, 20, 0], Similarity [0.67571807, 0.67570114, 0.6699429, 0.6

In [26]:
# Assuming you want to predict topics for the first 100 samples in 'df_Abstract_test'
num_samples_to_predict = 100

num_of_topics = 5
results = []

# Create an empty DataFrame to store the results
result_df_q = pd.DataFrame(columns=['query_publication_numbers', 'query_class_codes', 'query_abstract', 'query_predicted_topics'])

# Assuming you have a list of 100 queries in 'df_Abstract_test'
for i, (query, publication_number, abstract, class_codes) in enumerate(zip(df_Abstract_test['abstract'][:num_samples_to_predict], 
                                                    df_Abstract_test['publication_numbers'][:num_samples_to_predict],
                                                    df_Abstract_test['abstract'][:num_samples_to_predict],
                                                    df_Abstract_test['class_codes'][:num_samples_to_predict])):
    similar_topics, similarity = topic_model.find_topics(query, top_n=num_of_topics)
    results.append((similar_topics, similarity))
    
    # Store the results in the DataFrame
    result_df_q = result_df_q.append({
        'query_abstract': query,
        'query_predicted_topics': similar_topics,
        'query_publication_numbers': publication_number,
        'query_class_codes': class_codes
    }, ignore_index=True)
    
result_df_q


,query_publication_numbers,query_class_codes,query_abstract,query_predicted_topics
0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,"[40, 13, 10, 46, 7]"
1,US20230270344A1,"A61B5/02405,A61B5/01,A61B5/16,A61B5/02055,A61B...",A wearable monitoring device includes a band c...,"[13, 46, 39, 7, 10]"
2,US20230200909A1,"A61B17/17,A61B2090/061,A61B90/06,A61B2090/365,...",A number of improvements are provided relating...,"[39, 20, 3, -1, 0]"
3,US20230218347A1,"G06V10/46,G06V20/698,G06T2207/20112,G06T7/13,A...",Embodiments include a system for determining c...,"[36, 5, 7, 41, 34]"
4,US20230063013A1,"H04M1/72418,G08B,G08,G08B25/016,G,H04W4/023,G1...",A community based response system for providin...,"[13, 7, 10, 25, 26]"
...,...,...,...,...
95,US20230017310A1,"G06Q10/10,G,G06F,G06F16/90,G06F16/95,G16H30/20...",Features are disclosed for remote storage of m...,"[18, 32, 23, 12, -1]"
96,US20230091925A1,"H04L41/5061,G,G06,G06F,H04L67/00,H,H04,G06F16/...",Some of the embodiments herein provide a seaml...,"[5, -1, 32, 12, 18]"
97,US20230010638A1,"A61M5/142,A61,Y10S128/00,G06T2219/2016,G06F8/6...",A method and system is disclosed for operating...,"[16, 37, 1, -1, 2]"
98,US20230009812A1,"A61B5/14551,A61,G,A61B5/021,A61B5/746,A61B,A61...",A patient monitor including a physiological me...,"[10, 25, 13, 14, 7]"


In [27]:
# Check if the 'query_publication_numbers' field is unique
is_unique = result_df_q['query_publication_numbers'].nunique() == len(result_df_q)

if is_unique:
    print("The 'query_publication_numbers' field is unique.")
else:
    print("The 'query_publication_numbers' field is not unique.")


The 'query_publication_numbers' field is unique.


In [28]:
import pandas as pd

# Create an empty DataFrame to store the results
result_df = pd.DataFrame(columns=['publication_number', 'title', 'abstract', 'sub_classes', 'sub_class', 'topics', 'prob', 'query_publication_numbers', 'query_class_codes', 'query_abstract', 'query_predicted_topics'])

# Define the number of documents to retrieve for each topic
num_of_documents_to_retrieve = 10

for i, row in result_df_q.iterrows():
    query = row['query_abstract']
    predicted_topics = row['query_predicted_topics']
    
    for topic_id in predicted_topics:
        # Filter 'df_Abstract_topic' to get the top 'num_of_documents_to_retrieve' documents for the current topic_id
        topic_documents = df_Abstract_topic[df_Abstract_topic['topics'] == topic_id]
        
        # Sort the documents by probability in descending order
        topic_documents = topic_documents.sort_values(by='prob', ascending=False).head(num_of_documents_to_retrieve)
        
        # Append the results to the 'result_df' DataFrame
        for _, doc_row in topic_documents.iterrows():
            result_df = result_df.append({
                'query_publication_numbers': row['query_publication_numbers'],
                'query_class_codes': row['query_class_codes'],
                'query_abstract': query,
                'query_predicted_topics': [topic_id],  # Assign the current topic_id as a list
                'publication_number': doc_row['publication_number'],
                'title': doc_row['title'],
                'abstract': doc_row['abstract'],
                'sub_classes': doc_row['sub_classes'],
                'sub_class': doc_row['sub_class'],
                'topics': doc_row['topics'],
                'prob': doc_row['prob']
            }, ignore_index=True)

# Now, 'result_df' contains the top 10 most probable documents for each predicted topic list for each query
result_df


,publication_number,title,abstract,sub_classes,sub_class,topics,prob,query_publication_numbers,query_class_codes,query_abstract,query_predicted_topics
0,US2020145183A1,"Physiological information system, physiologica...",A physiological information system includes: a...,"['G16H40/67', 'G16H40/67']","['G16H', 'G16H']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40]
1,US2022167894A1,"Physiological state control apparatus, physiol...",A physiological state control apparatus includ...,"['A61B5/16', 'G16H50/30']","['A61B', 'G16H']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40]
2,US2010222650A1,Biological signal measuring apparatus,A biological signal measuring apparatus includ...,"['G16H20/40', 'A61B5/74']","['G16H', 'A61B']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40]
3,US2010211404A1,Methods and Systems for Real-Time RRi Values P...,Biological outcomes are detected and/or predic...,"['G16H10/60', 'A61B5/316']","['G16H', 'A61B']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40]
4,US2009187392A1,System and method for monitoring a health cond...,An ingestible bolus may be disposed within the...,"['G16H40/67', 'A61B5/02055']","['G16H', 'A61B']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40]
...,...,...,...,...,...,...,...,...,...,...,...
4995,US2021390595A1,Electronic reputation management,At least one computer-readable medium on which...,"['G06Q30/02', 'G16H40/20']","['G06Q', 'G16H']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3]
4996,US2021398627A1,Systems and methods for health information mes...,Messages having patient healthcare information...,"['G16H40/67', 'H04L67/01']","['G16H', 'H04L']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3]
4997,US2021407679A1,Deep-learning based certainty qualification in...,A method for assessing diagnostic certainty in...,"['G16H15/00', 'G06F40/253']","['G16H', 'G06F']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3]
4998,US2021407683A1,Method and system for remote health monitoring...,"A method, a system, and a non-transitory stora...","['G16H50/20', 'G16H50/30']","['G16H', 'G16H']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3]


In [29]:
result_df['query_class_codes'][0]

'A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/7221,A61B5/14532,G16H40/00,A61B5/14546,G16H40/60,A61B5/68,A61B2562/08,H,A61B1/00,A61B2562/221,A61B5/7475,A61B2562/18,A61B5/746,A61B5/7405,H05K,G16H40/67,A61B,Y10,A61B5/6813,Y,A61B2562/185,G16H10/40,A61B5/6826,A61B5/02,A61B5/0002,A61B5/1455,A61B5/6832,A,A61B5/7235,A61B5/6801,A61B5/7278,G16H,A61B5/026,A61B5/742,A61B2562/22,A61B5/024,G16,A61B5/02416,A61B5/683,A61B5/0015,A61B5/6825,Y10S,A61B5/1495,A61B5/72,A61B5/0261,A61B2562/085,A61B2562/222,A61B5/7275,A61B5/00,A61B5/145,A61B5/74,G16H10/00,H05K999/99,A61B5/7246,Y10S439/909,A61B5/14552,A61B5/6838,A61B5/6815,A61B5/7271,H05K999/00,A61B5/02427,A61B5/0205,Y10S439/00,A61B5/0022,A61B5/6814,A61B2562/00,H05'

In [30]:
# Add a new column to store the filtered codes
result_df['query_codes_G16H'] = ''

# Define a function to extract codes starting with 'G61H' from the class codes
def extract_G16H_codes(class_codes):
    codes = class_codes.split(',')
    return ','.join([code for code in codes if code.startswith('G16H')])

# Iterate through rows and update the 'query_codes_G61H' column
for index, row in result_df.iterrows():
    class_codes = row['query_class_codes']
    filtered_codes = extract_G16H_codes(class_codes)
    result_df.at[index, 'query_codes_G16H'] = filtered_codes

# Display the updated DataFrame
result_df


,publication_number,title,abstract,sub_classes,sub_class,topics,prob,query_publication_numbers,query_class_codes,query_abstract,query_predicted_topics,query_codes_G16H
0,US2020145183A1,"Physiological information system, physiologica...",A physiological information system includes: a...,"['G16H40/67', 'G16H40/67']","['G16H', 'G16H']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G..."
1,US2022167894A1,"Physiological state control apparatus, physiol...",A physiological state control apparatus includ...,"['A61B5/16', 'G16H50/30']","['A61B', 'G16H']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G..."
2,US2010222650A1,Biological signal measuring apparatus,A biological signal measuring apparatus includ...,"['G16H20/40', 'A61B5/74']","['G16H', 'A61B']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G..."
3,US2010211404A1,Methods and Systems for Real-Time RRi Values P...,Biological outcomes are detected and/or predic...,"['G16H10/60', 'A61B5/316']","['G16H', 'A61B']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G..."
4,US2009187392A1,System and method for monitoring a health cond...,An ingestible bolus may be disposed within the...,"['G16H40/67', 'A61B5/02055']","['G16H', 'A61B']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G..."
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,US2021390595A1,Electronic reputation management,At least one computer-readable medium on which...,"['G06Q30/02', 'G16H40/20']","['G06Q', 'G16H']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3],"G16H,G16H10/00,G16H40/20,G16H40/00,G16H10/60"
4996,US2021398627A1,Systems and methods for health information mes...,Messages having patient healthcare information...,"['G16H40/67', 'H04L67/01']","['G16H', 'H04L']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3],"G16H,G16H10/00,G16H40/20,G16H40/00,G16H10/60"
4997,US2021407679A1,Deep-learning based certainty qualification in...,A method for assessing diagnostic certainty in...,"['G16H15/00', 'G06F40/253']","['G16H', 'G06F']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3],"G16H,G16H10/00,G16H40/20,G16H40/00,G16H10/60"
4998,US2021407683A1,Method and system for remote health monitoring...,"A method, a system, and a non-transitory stora...","['G16H50/20', 'G16H50/30']","['G16H', 'G16H']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3],"G16H,G16H10/00,G16H40/20,G16H40/00,G16H10/60"


In [32]:
# Create a new column to store the common code
import re
result_df['exact_match_code'] = ''

# Iterate through rows and compare 'sub_classes' and 'query_codes_G16H'
for index, row in result_df.iterrows():
    sub_classes_str = row['sub_classes']  # Data format in this field "['A61B5/00', 'G16H40/67']"
    query_codes_G16H = row['query_codes_G16H']  # Data format in this field 'G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H'
    
    # Custom extraction of values from sub_classes_str
    sub_classes = re.findall(r"'([^']*)'", sub_classes_str)
    
    # Split the codes into lists
    sub_class_list = [code.strip() for code in sub_classes]
    query_codes_list = query_codes_G16H.split(',')
    
    # Check for common codes
    exact_match_code = [code for code in sub_class_list if code in query_codes_list]
    
    # Join the common codes into a single string
    exact_match_code_str = ','.join(exact_match_code)
    
    # Update the 'exact_match_code' column with the exact_match_code
    result_df.at[index, 'exact_match_code'] = exact_match_code_str

# Display the updated DataFrame
result_df


,publication_number,title,abstract,sub_classes,sub_class,topics,prob,query_publication_numbers,query_class_codes,query_abstract,query_predicted_topics,query_codes_G16H,exact_match_code
0,US2020145183A1,"Physiological information system, physiologica...",A physiological information system includes: a...,"['G16H40/67', 'G16H40/67']","['G16H', 'G16H']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G...","G16H40/67,G16H40/67"
1,US2022167894A1,"Physiological state control apparatus, physiol...",A physiological state control apparatus includ...,"['A61B5/16', 'G16H50/30']","['A61B', 'G16H']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G...",
2,US2010222650A1,Biological signal measuring apparatus,A biological signal measuring apparatus includ...,"['G16H20/40', 'A61B5/74']","['G16H', 'A61B']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G...",
3,US2010211404A1,Methods and Systems for Real-Time RRi Values P...,Biological outcomes are detected and/or predic...,"['G16H10/60', 'A61B5/316']","['G16H', 'A61B']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G...",
4,US2009187392A1,System and method for monitoring a health cond...,An ingestible bolus may be disposed within the...,"['G16H40/67', 'A61B5/02055']","['G16H', 'A61B']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G...",G16H40/67
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,US2021390595A1,Electronic reputation management,At least one computer-readable medium on which...,"['G06Q30/02', 'G16H40/20']","['G06Q', 'G16H']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3],"G16H,G16H10/00,G16H40/20,G16H40/00,G16H10/60",G16H40/20
4996,US2021398627A1,Systems and methods for health information mes...,Messages having patient healthcare information...,"['G16H40/67', 'H04L67/01']","['G16H', 'H04L']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3],"G16H,G16H10/00,G16H40/20,G16H40/00,G16H10/60",
4997,US2021407679A1,Deep-learning based certainty qualification in...,A method for assessing diagnostic certainty in...,"['G16H15/00', 'G06F40/253']","['G16H', 'G06F']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3],"G16H,G16H10/00,G16H40/20,G16H40/00,G16H10/60",
4998,US2021407683A1,Method and system for remote health monitoring...,"A method, a system, and a non-transitory stora...","['G16H50/20', 'G16H50/30']","['G16H', 'G16H']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3],"G16H,G16H10/00,G16H40/20,G16H40/00,G16H10/60",


In [ ]:
# result_df.sample(n=100)

In [33]:
# Calculate the count of 'exact_match_code' for each group and assign it to all rows within the group
result_df['count_exact_match_top50'] = result_df.groupby('query_publication_numbers')['exact_match_code'].transform(lambda x: x[x != ''].count())
result_df

,publication_number,title,abstract,sub_classes,sub_class,topics,prob,query_publication_numbers,query_class_codes,query_abstract,query_predicted_topics,query_codes_G16H,exact_match_code,count_exact_match_top50
0,US2020145183A1,"Physiological information system, physiologica...",A physiological information system includes: a...,"['G16H40/67', 'G16H40/67']","['G16H', 'G16H']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G...","G16H40/67,G16H40/67",14
1,US2022167894A1,"Physiological state control apparatus, physiol...",A physiological state control apparatus includ...,"['A61B5/16', 'G16H50/30']","['A61B', 'G16H']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G...",,14
2,US2010222650A1,Biological signal measuring apparatus,A biological signal measuring apparatus includ...,"['G16H20/40', 'A61B5/74']","['G16H', 'A61B']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G...",,14
3,US2010211404A1,Methods and Systems for Real-Time RRi Values P...,Biological outcomes are detected and/or predic...,"['G16H10/60', 'A61B5/316']","['G16H', 'A61B']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G...",,14
4,US2009187392A1,System and method for monitoring a health cond...,An ingestible bolus may be disposed within the...,"['G16H40/67', 'A61B5/02055']","['G16H', 'A61B']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G...",G16H40/67,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,US2021390595A1,Electronic reputation management,At least one computer-readable medium on which...,"['G06Q30/02', 'G16H40/20']","['G06Q', 'G16H']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3],"G16H,G16H10/00,G16H40/20,G16H40/00,G16H10/60",G16H40/20,21
4996,US2021398627A1,Systems and methods for health information mes...,Messages having patient healthcare information...,"['G16H40/67', 'H04L67/01']","['G16H', 'H04L']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3],"G16H,G16H10/00,G16H40/20,G16H40/00,G16H10/60",,21
4997,US2021407679A1,Deep-learning based certainty qualification in...,A method for assessing diagnostic certainty in...,"['G16H15/00', 'G06F40/253']","['G16H', 'G06F']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3],"G16H,G16H10/00,G16H40/20,G16H40/00,G16H10/60",,21
4998,US2021407683A1,Method and system for remote health monitoring...,"A method, a system, and a non-transitory stora...","['G16H50/20', 'G16H50/30']","['G16H', 'G16H']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3],"G16H,G16H10/00,G16H40/20,G16H40/00,G16H10/60",,21


In [34]:
import pandas as pd


# Initialize RFR as NaN
result_df['RFR'] = float('nan')

# Create a dictionary to store the first occurrence index for each group
first_occurrence_indices = {}

# Iterate through the DataFrame
for index, row in result_df.iterrows():
    query_pub_num = row['query_publication_numbers']
    exact_match_code = row['exact_match_code']

    # Check if the exact_match_code is not empty and it's the first occurrence within the group
    if exact_match_code != '' and not pd.isna(exact_match_code) and query_pub_num not in first_occurrence_indices:
        first_occurrence_indices[query_pub_num] = index

# Calculate RFR for each group based on the first occurrence within each group
for query_pub_num, first_occurrence_index in first_occurrence_indices.items():
    group_indices = result_df[result_df['query_publication_numbers'] == query_pub_num].index
    result_df.loc[group_indices, 'RFR'] = first_occurrence_index - group_indices[0] + 1

# Print the updated DataFrame
result_df

,publication_number,title,abstract,sub_classes,sub_class,topics,prob,query_publication_numbers,query_class_codes,query_abstract,query_predicted_topics,query_codes_G16H,exact_match_code,count_exact_match_top50,RFR
0,US2020145183A1,"Physiological information system, physiologica...",A physiological information system includes: a...,"['G16H40/67', 'G16H40/67']","['G16H', 'G16H']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G...","G16H40/67,G16H40/67",14,1.0
1,US2022167894A1,"Physiological state control apparatus, physiol...",A physiological state control apparatus includ...,"['A61B5/16', 'G16H50/30']","['A61B', 'G16H']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G...",,14,1.0
2,US2010222650A1,Biological signal measuring apparatus,A biological signal measuring apparatus includ...,"['G16H20/40', 'A61B5/74']","['G16H', 'A61B']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G...",,14,1.0
3,US2010211404A1,Methods and Systems for Real-Time RRi Values P...,Biological outcomes are detected and/or predic...,"['G16H10/60', 'A61B5/316']","['G16H', 'A61B']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G...",,14,1.0
4,US2009187392A1,System and method for monitoring a health cond...,An ingestible bolus may be disposed within the...,"['G16H40/67', 'A61B5/02055']","['G16H', 'A61B']",40,1.0,US20230238130A1,"A61B5/14551,A61,A61B5/0295,A61B5/6829,G,A61B5/...",A physiological sensor has light emitting sour...,[40],"G16H40/00,G16H40/60,G16H40/67,G16H10/40,G16H,G...",G16H40/67,14,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,US2021390595A1,Electronic reputation management,At least one computer-readable medium on which...,"['G06Q30/02', 'G16H40/20']","['G06Q', 'G16H']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3],"G16H,G16H10/00,G16H40/20,G16H40/00,G16H10/60",G16H40/20,21,1.0
4996,US2021398627A1,Systems and methods for health information mes...,Messages having patient healthcare information...,"['G16H40/67', 'H04L67/01']","['G16H', 'H04L']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3],"G16H,G16H10/00,G16H40/20,G16H40/00,G16H10/60",,21,1.0
4997,US2021407679A1,Deep-learning based certainty qualification in...,A method for assessing diagnostic certainty in...,"['G16H15/00', 'G06F40/253']","['G16H', 'G06F']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3],"G16H,G16H10/00,G16H40/20,G16H40/00,G16H10/60",,21,1.0
4998,US2021407683A1,Method and system for remote health monitoring...,"A method, a system, and a non-transitory stora...","['G16H50/20', 'G16H50/30']","['G16H', 'G16H']",3,1.0,US20230138516A1,"G16H,G16H10/00,G06Q10/1093,G06Q10/10,G16,G06Q1...",A system to manage records in a healthcare pra...,[3],"G16H,G16H10/00,G16H40/20,G16H40/00,G16H10/60",,21,1.0


In [35]:
# Filter the DataFrame to keep only the top 5 records within each group
result_df_filter_top5 = result_df.groupby('query_publication_numbers').apply(lambda x: x.head(5)).reset_index(drop=True)

# Count the occurrences of count_exact_match if it has data
result_df_filter_top5['count_exact_match_count'] = result_df_filter_top5['exact_match_code'].apply(lambda x: 1 if x != '' else 0)

# Calculate the total count for each group
result_df_filter_top5['count_exact_match_top5'] = result_df_filter_top5.groupby('query_publication_numbers')['count_exact_match_count'].transform('sum')
df_top5=result_df_filter_top5[["publication_number","title","abstract","sub_classes","sub_class","topics","prob", "query_publication_numbers","query_abstract","query_codes_G16H","exact_match_code","count_exact_match_top50","count_exact_match_top5"]]
df_top5

,publication_number,title,abstract,sub_classes,sub_class,topics,prob,query_publication_numbers,query_abstract,query_codes_G16H,exact_match_code,count_exact_match_top50,count_exact_match_top5
0,US2016213261A1,Method and device for improving prediction and...,"Devices, methods and systems for predicting an...","['A61B5/0205', 'G16H50/20']","['A61B', 'G16H']",10,1.0,US20230001090A1,Methods and systems for delaying alarms that i...,"G16H20/17,G16H40/00,G16H40/60,G16H40/67,G16H20...",,23,3
1,US2004236188A1,Method and apparatus for monitoring using a ma...,A method and an apparatus for use in monitorin...,"['G16H50/50', 'G16H15/00']","['G16H', 'G16H']",10,1.0,US20230001090A1,Methods and systems for delaying alarms that i...,"G16H20/17,G16H40/00,G16H40/60,G16H40/67,G16H20...",G16H50/50,23,3
2,US2021202061A1,Synchronizing physiological measurement data s...,There is provided a method in a collector devi...,"['G06F1/10', 'G16H20/30']","['G06F', 'G16H']",10,1.0,US20230001090A1,Methods and systems for delaying alarms that i...,"G16H20/17,G16H40/00,G16H40/60,G16H40/67,G16H20...",,23,3
3,US2010286490A1,Interactive patient monitoring system using sp...,"Systems, methods and techniques are described ...","['G10L15/00', 'G16H40/67']","['G10L', 'G16H']",10,1.0,US20230001090A1,Methods and systems for delaying alarms that i...,"G16H20/17,G16H40/00,G16H40/60,G16H40/67,G16H20...",G16H40/67,23,3
4,US2021161393A1,Temperature measurement,An apparatus is provided for receiving an indi...,"['G16H40/60', 'G01K13/20']","['G16H', 'G01K']",10,1.0,US20230001090A1,Methods and systems for delaying alarms that i...,"G16H20/17,G16H40/00,G16H40/60,G16H40/67,G16H20...",G16H40/60,23,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,US2010198083A1,Random Body Movement Cancellation for Non-Cont...,A method and system for cancelling body moveme...,"['A61B5/0205', 'G16H40/67']","['A61B', 'G16H']",13,1.0,US20230270344A1,A wearable monitoring device includes a band c...,"G16H40/00,G16H40/67,G16H20/30,G16H30/40,G16H30...",G16H40/67,29,4
496,US2019239800A1,Systems and Methods for Detecting and Characte...,Various systems are disclosed herein for detec...,"['A61B5/11', 'G16H15/00']","['A61B', 'G16H']",13,1.0,US20230270344A1,A wearable monitoring device includes a band c...,"G16H40/00,G16H40/67,G16H20/30,G16H30/40,G16H30...",,29,4
497,US2013278414A1,Biometric attribute anomoly detection system w...,"A system, methods and server for monitoring he...","['G16H40/67', 'G08B21/02']","['G16H', 'G08B']",13,1.0,US20230270344A1,A wearable monitoring device includes a band c...,"G16H40/00,G16H40/67,G16H20/30,G16H30/40,G16H30...",G16H40/67,29,4
498,US2021217531A1,Cloud-connected ring-based sensor system,A cloud-connected ring-based sensor system inc...,"['G16H50/20', 'A61B2560/0209']","['G16H', 'A61B']",13,1.0,US20230270344A1,A wearable monitoring device includes a band c...,"G16H40/00,G16H40/67,G16H20/30,G16H30/40,G16H30...",G16H50/20,29,4


In [36]:
filtered_df = result_df[result_df['query_publication_numbers'] == 'US20230001090A1']
filtered_df

,publication_number,title,abstract,sub_classes,sub_class,topics,prob,query_publication_numbers,query_class_codes,query_abstract,query_predicted_topics,query_codes_G16H,exact_match_code,count_exact_match_top50,RFR
3900,US2016213261A1,Method and device for improving prediction and...,"Devices, methods and systems for predicting an...","['A61B5/0205', 'G16H50/20']","['A61B', 'G16H']",10,1.0,US20230001090A1,"A61M5/142,A61,G08B,G08,G16H20/17,A61B5/4836,G,...",Methods and systems for delaying alarms that i...,[10],"G16H20/17,G16H40/00,G16H40/60,G16H40/67,G16H20...",,23,2.0
3901,US2004236188A1,Method and apparatus for monitoring using a ma...,A method and an apparatus for use in monitorin...,"['G16H50/50', 'G16H15/00']","['G16H', 'G16H']",10,1.0,US20230001090A1,"A61M5/142,A61,G08B,G08,G16H20/17,A61B5/4836,G,...",Methods and systems for delaying alarms that i...,[10],"G16H20/17,G16H40/00,G16H40/60,G16H40/67,G16H20...",G16H50/50,23,2.0
3902,US2021202061A1,Synchronizing physiological measurement data s...,There is provided a method in a collector devi...,"['G06F1/10', 'G16H20/30']","['G06F', 'G16H']",10,1.0,US20230001090A1,"A61M5/142,A61,G08B,G08,G16H20/17,A61B5/4836,G,...",Methods and systems for delaying alarms that i...,[10],"G16H20/17,G16H40/00,G16H40/60,G16H40/67,G16H20...",,23,2.0
3903,US2010286490A1,Interactive patient monitoring system using sp...,"Systems, methods and techniques are described ...","['G10L15/00', 'G16H40/67']","['G10L', 'G16H']",10,1.0,US20230001090A1,"A61M5/142,A61,G08B,G08,G16H20/17,A61B5/4836,G,...",Methods and systems for delaying alarms that i...,[10],"G16H20/17,G16H40/00,G16H40/60,G16H40/67,G16H20...",G16H40/67,23,2.0
3904,US2021161393A1,Temperature measurement,An apparatus is provided for receiving an indi...,"['G16H40/60', 'G01K13/20']","['G16H', 'G01K']",10,1.0,US20230001090A1,"A61M5/142,A61,G08B,G08,G16H20/17,A61B5/4836,G,...",Methods and systems for delaying alarms that i...,[10],"G16H20/17,G16H40/00,G16H40/60,G16H40/67,G16H20...",G16H40/60,23,2.0
3905,US2010317931A1,"Method, Device and Computer Program Product fo...","A method, device, and computer program product...","['G16H40/63', 'A61B5/369']","['G16H', 'A61B']",10,1.0,US20230001090A1,"A61M5/142,A61,G08B,G08,G16H20/17,A61B5/4836,G,...",Methods and systems for delaying alarms that i...,[10],"G16H20/17,G16H40/00,G16H40/60,G16H40/67,G16H20...",,23,2.0
3906,US2018225417A1,Cognitive notification for mental support,Systems and methods are disclosed for detectin...,"['A61B5/16', 'G16H50/70']","['A61B', 'G16H']",10,1.0,US20230001090A1,"A61M5/142,A61,G08B,G08,G16H20/17,A61B5/4836,G,...",Methods and systems for delaying alarms that i...,[10],"G16H20/17,G16H40/00,G16H40/60,G16H40/67,G16H20...",,23,2.0
3907,US2018168463A1,Exercise triggered cardiovascular pressure mea...,"Systems, devices disclosed provide example met...","['A61B5/00', 'G16H20/30']","['A61B', 'G16H']",10,1.0,US20230001090A1,"A61M5/142,A61,G08B,G08,G16H20/17,A61B5/4836,G,...",Methods and systems for delaying alarms that i...,[10],"G16H20/17,G16H40/00,G16H40/60,G16H40/67,G16H20...",,23,2.0
3908,US2018132776A1,Systems and methods for estimating and predict...,Systems and methods for estimating emotional s...,"['A61B5/024', 'G16H50/20']","['A61B', 'G16H']",10,1.0,US20230001090A1,"A61M5/142,A61,G08B,G08,G16H20/17,A61B5/4836,G,...",Methods and systems for delaying alarms that i...,[10],"G16H20/17,G16H40/00,G16H40/60,G16H40/67,G16H20...",,23,2.0
3909,US2018008208A1,Patient-Assisted Alarm System,"In the present invention, a system and associa...","['A61B5/026', 'G16H40/63']","['A61B', 'G16H']",10,1.0,US20230001090A1,"A61M5/142,A61,G08B,G08,G16H20/17,A61B5/4836,G,...",Methods and systems for delaying alarms that i...,[10],"G16H20/17,G16H40/00,G16H40/60,G16H40/67,G16H20...",,23,2.0


In [37]:
result_df.to_csv('BERTopic-without-Preprocessing-Abstract.csv', index=False)

In [38]:
average_count_exact_match_top5 = df_top5.groupby('query_publication_numbers')['count_exact_match_top5'].mean().reset_index()
average_count_exact_match_top5


,query_publication_numbers,count_exact_match_top5
0,US20230001090A1,3.0
1,US20230001263A1,3.0
2,US20230005591A1,3.0
3,US20230005611A1,1.0
4,US20230009812A1,1.0
...,...,...
95,US20230255518A1,0.0
96,US20230259481A1,3.0
97,US20230260617A1,3.0
98,US20230264029A1,0.0


In [39]:
average_count_exact_match_top5 = average_count_exact_match_top5['count_exact_match_top5'].mean()
print("Average of count_exact_match_top5:", average_count_exact_match_top5)

Average of count_exact_match_top5: 1.95


In [40]:
average_count_exact_match_top50 = result_df.groupby('query_publication_numbers')['count_exact_match_top50'].mean().reset_index()
average_count_exact_match_top50

,query_publication_numbers,count_exact_match_top50
0,US20230001090A1,23.0
1,US20230001263A1,11.0
2,US20230005591A1,19.0
3,US20230005611A1,4.0
4,US20230009812A1,7.0
...,...,...
95,US20230255518A1,11.0
96,US20230259481A1,16.0
97,US20230260617A1,16.0
98,US20230264029A1,5.0


In [41]:
average_count_exact_match_top50 = average_count_exact_match_top50['count_exact_match_top50'].mean()
print("Average of count_exact_match_top50:", average_count_exact_match_top50)

Average of count_exact_match_top50: 14.45


In [42]:
avg_RFR =  result_df.groupby('query_publication_numbers')['RFR'].mean().reset_index()
avg_RFR

,query_publication_numbers,RFR
0,US20230001090A1,2.0
1,US20230001263A1,2.0
2,US20230005591A1,2.0
3,US20230005611A1,5.0
4,US20230009812A1,1.0
...,...,...
95,US20230255518A1,6.0
96,US20230259481A1,3.0
97,US20230260617A1,1.0
98,US20230264029A1,6.0


In [43]:
avg_RFR = avg_RFR['RFR'].mean()
print("Average of RFR:", avg_RFR)

Average of RFR: 3.783505154639175
